In [1]:
from cnn_model import CNN
model = CNN(1e-5)

/home/jovyan/my-conda-envs/RNAmodEnv/lib/python3.8/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: /home/jovyan/my-conda-envs/RNAmodEnv/lib/python3.8/site-packages/torchvision/image.so: undefined symbol: _ZN3c106detail19maybe_wrap_dim_slowEllb
  warn(f"Failed to load image Python extension: {e}")


In [2]:
from dataloading import get_demo_dataset, get_my_dataset

# train_dset, valid_dset = get_demo_dataset(valid_limit=1000)
train_dset, valid_dset = get_my_dataset(valid_limit=10000, valid_files_count=10)

valid files indicies
[15, 46, 109, 112, 138, 145, 162, 200, 210, 224]
[74, 79, 111, 144, 184, 251, 295, 316, 473, 497]


In [ ]:
# %%time
from torch.utils.data import DataLoader
import pytorch_lightning as pl
from pytorch_lightning.loggers import CometLogger
#TODO more workers? bigger batch size?
train_loader = DataLoader(train_dset, batch_size=512, num_workers=32,
                          pin_memory=True, persistent_workers=True)
valid_loader = DataLoader(valid_dset, batch_size=256, num_workers=32,
                          pin_memory=True, persistent_workers=True)


logger = CometLogger(api_key="TEVQbgxxvilM1WdTyqZLJ57ac", project_name='RNAModif')
trainer= pl.Trainer(
    max_steps = 100000, logger=logger, accelerator='gpu', #max_epochs=-1,
    auto_lr_find=False, val_check_interval=500, #500 
    log_every_n_steps=500, benchmark=True, precision=16)#, profiler="simple")
trainer.fit(model, train_loader, valid_loader)



CometLogger will be initialized in online mode
Using 16bit native Automatic Mixed Precision (AMP)
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name | Type       | Params
------------------------------------
0 | net  | Sequential | 495 K 
1 | acc  | Accuracy   | 0     
------------------------------------
495 K     Trainable params
0         Non-trainable params
495 K     Total params
0.990     Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

COMET WARNING: Comet has disabled auto-logging functionality as it has been imported after the following ML modules: torch. Metrics and hyperparameters can still be logged using comet_ml.log_metrics() and comet_ml.log_parameters()
COMET WARNING: As you are running in a Jupyter environment, you will need to call `experiment.end()` when finished to ensure all metrics and code are logged before exiting.
COMET INFO: Experiment is live on comet.ml https://www.comet.com/vlasta/rnamodif/66706d3332d44e9393c410f3566d6953



Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

In [4]:
import torch
torch.save(trainer.model.state_dict(), './saved_models/test_model')

In [5]:
the_model = model = CNN(1e-4)
the_model.load_state_dict(torch.load('./saved_models/test_model'))


<All keys matched successfully>

In [6]:
import captum
from captum.attr import LayerIntegratedGradients, TokenReferenceBase, visualization
from captum.attr import IntegratedGradients
from matplotlib import pyplot as plt
import seaborn as sns

#TODO integrate basecaller + compute how much attention does my model pay to Us vs other bases
# this could be problematic - signal represents 5 nucleotides, what is its stride?
# If Us are everywhere, could be hard to pinpoint attributions

x, y = next(iter(valid_loader))
inp = torch.tensor(x)[0:1,:,:]
baseline = torch.zeros(inp.shape)

ig = IntegratedGradients(the_model)
attributions, delta = ig.attribute(inp, baseline, target=0, return_convergence_delta=True)
# print('IG Attributions:', attributions.shape)
# print('Convergence Delta:', delta.shape)

figure_name = plt.figure(figsize=(15, 3))
signal = x[0,0,:].numpy()
signal_norm = (signal/(2*np.max(np.abs(signal))))+0.5

plt.plot(signal_norm, zorder=1, color='black')
heatmap = sns.heatmap(attributions[0,:,:], zorder=0,cbar=True, cmap='seismic_r', center=0)

plt.show()

RuntimeError: DataLoader worker (pid(s) 751936, 752064, 752065, 752193, 752448, 752449, 752704, 752705, 752833, 752834, 753216, 753344, 753345, 753547, 753601, 753602, 753857, 753858, 753991, 754241, 754242, 754243, 754478, 754809, 754881, 755106, 755137, 755138, 755520, 755521, 755649, 755650) exited unexpectedly